In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import h5py

from mpl_toolkits.mplot3d import Axes3D

import sys
sys.path.append("/home/habjan.e/TNG/TNG_cluster_dynamics")
import TNG_DA

### Pick a cluster index

In [ ]:
cluster_ind = 100

### Import subhalo information

In [ ]:
pos, vel, groups, subhalo_masses, h, halo_mass = TNG_DA.get_cluster_props(cluster_ind)

### Import particle data

In [ ]:
fName = f'/home/habjan.e/TNG/Data/TNG_data/halo_cutouts_dm_{cluster_ind}.hdf5'

with h5py.File(fName, 'r') as f:

    part_coordinates = f['PartType1']['Coordinates'][:]
    velocities = f['PartType1']['Velocities'][:]
    ids = f['PartType1']['ParticleIDs'][:]

masses = np.zeros(part_coordinates.shape[0]) + 5.9 * 10**7

coordinates = TNG_DA.coord_cm_corr(cluster_ind = cluster_ind, coordinates = part_coordinates)

### Import ROCKSTAR pandas df

In [ ]:
output_path = f'/home/habjan.e/TNG/Data/rockstar_output/rockstar_subhalos_{cluster_ind}.list'


column_names = ["id", "pos_0", "pos_1", "pos_2", "pos_3", "pos_4", "pos_5", "bulkvel_0", "bulkvel_1", "bulkvel_2", "corevel_0", "corevel_1", "corevel_2", "mass_grav", "p_start", "num_p"]
df = pd.read_csv(output_path, sep=r"\s+", comment="#", names=column_names)
df

### Add member lists to pandas df

In [ ]:
members_path = f'/home/habjan.e/TNG/Data/rockstar_output/rockstar_subhalo_members_{cluster_ind}.list'

members = pd.read_csv(members_path,
                      sep=r"\s+", names=["halo_id","particle_id"])

member_id = np.array(members['particle_id'])
mem_halo_id = np.array(members['halo_id'])

grouped = members.groupby("halo_id")["particle_id"].apply(list)
df["member_ids"] = df["id"].map(grouped)
len(df.loc[np.where((df['num_p'] > 10**3) & (df['num_p'] < 10**4))[0]])

### Plot Some subhalos

In [ ]:
fig = plt.figure()
ax  = fig.add_subplot(111, projection='3d')

for i in np.where((df['num_p'] > 10**3) & (df['num_p'] < 10**4))[0]:

    plot_bool = np.isin(ids, df['member_ids'][i])

    ax.scatter(
        coordinates[plot_bool,1],  # x
        coordinates[plot_bool,2],  # y
        coordinates[plot_bool,0],  # z
        s=2              
    )

# label axes
ax.set_xlabel('Y [kpc]')
ax.set_ylabel('Z [kpc]')
ax.set_zlabel('X [kpc]')

plt.tight_layout()
plt.show()

### Do a distance calculation to find which subhalo is closest to particles

In [ ]:
k = 5000

sub_mem = np.zeros(pos.shape[0])

real_subs = np.array(df['id'][np.where((df['num_p'] > 10**3) & (df['num_p'] < 10**4))[0]])
real_bool = np.isin(mem_halo_id, real_subs)

for i in range(pos.shape[0]):

    dist = np.sqrt((pos[i, 0] - coordinates[real_bool, 0])**2 + (pos[i, 1] - coordinates[real_bool, 1])**2 + (pos[i, 2] - coordinates[real_bool, 2])**2)

    idx = np.argpartition(dist, k-1)[:k]

    k_ids = ids[real_bool][idx]

    sub_mem[i] = np.unique(mem_halo_id[real_bool][np.isin(member_id[real_bool], k_ids)])[0]

### Plot the found subhalos in their respective substructure

In [ ]:
fig = plt.figure()
ax  = fig.add_subplot(111, projection='3d')

for i in np.unique(sub_mem):

    plot_bool = np.where(i == sub_mem)[0]

    ax.scatter(
        pos[plot_bool,1],  # x
        pos[plot_bool,2],  # y
        pos[plot_bool,0],  # z
        s=100              
    )

# label axes
ax.set_xlabel('Y [kpc]')
ax.set_ylabel('Z [kpc]')
ax.set_zlabel('X [kpc]')

plt.tight_layout()
plt.show()

### The same plot but in velocity space

In [ ]:
fig = plt.figure()
ax  = fig.add_subplot(111, projection='3d')

for i in np.unique(sub_mem):

    plot_bool = np.where(i == sub_mem)[0]

    ax.scatter(
        vel[plot_bool,2],  # x
        vel[plot_bool,0],  # y
        vel[plot_bool,1],  # z
        s=100              
    )

# label axes
ax.set_xlabel(r'$v_{z}$ [$\frac{km}{s}$]')
ax.set_ylabel(r'$v_{x}$ [$\frac{km}{s}$]')
ax.set_zlabel(r'$v_{y}$ [$\frac{km}{s}$]')

plt.tight_layout()
plt.show()